# 1. 生成器

## 1.1 列表推导式

## 1.2 生成器表达式

## 1.3 生成器函数

# 2. 迭代器

# 3. 函数式编程

## 3.1 高阶函数

## 3.2 内置高阶函数

## 3.3 闭包

## 3.4 装饰器

## 3.5 匿名函数

## 3.6 偏函数

# 4. 面向对象编程

## 4.1 类与对象

## 4.2 访问限制

## 4.3 面向对象三大特性

### 4.3.1 继承

### 4.3.2 封装

### 4.3.3 多态

## 4.4 内置方法

## 4.5 元类